In [5]:
import os
import pandas as pd
from datetime import datetime, timedelta
from langfuse import Langfuse
from dotenv import load_dotenv

load_dotenv()

langfuse = Langfuse()

In [6]:
x = langfuse.api.trace.get("3c7300a2c603e23e4cd87f07cdc5c34f")

In [37]:
x.dict()

{'id': '3c7300a2c603e23e4cd87f07cdc5c34f',
 'timestamp': datetime.datetime(2025, 11, 20, 9, 28, 20, 473000, tzinfo=datetime.timezone.utc),
 'name': 'call_llm',
 'input': {'model': 'azure/gpt-4o-mini',
  'config': {'system_instruction': '\n        **Role:** Host agent for the agent-to-agent protocol; delegates queries to specialized remote agents with maximum efficiency.\n\n**Core Directives:**\n\n* **Task Delegation:** Use the `send_message` function to assign precise, actionable tasks to remote agents.\n* **Full Context Provision:** If an agent repeatedly asks for user confirmation, it likely lacks conversation history. Include all relevant context in the task to prevent this.\n* **Autonomous Multi-Agent Engagement:** Engage any required agents directly—never seek user permission or preference. If multiple agents are needed, orchestrate them seamlessly.\n* **Intelligent Inter-Agent Collaboration:** Instruct agents to determine if they need data from another agent. **If Agent A says "I

In [33]:
# y = x.dict()
import json
y = json.dump(x)

TypeError: dump() missing 1 required positional argument: 'fp'

In [8]:
context = "following is the system instructions of agent named : routing agent\n "+y["input"]["config"]["system_instruction"]
lines = context.split("\n")
print(lines)
print(context)

['following is the system instructions of agent named : routing agent', ' ', '        **Role:** Host agent for the agent-to-agent protocol; delegates queries to specialized remote agents with maximum efficiency.', '', '**Core Directives:**', '', '* **Task Delegation:** Use the `send_message` function to assign precise, actionable tasks to remote agents.', '* **Full Context Provision:** If an agent repeatedly asks for user confirmation, it likely lacks conversation history. Include all relevant context in the task to prevent this.', '* **Autonomous Multi-Agent Engagement:** Engage any required agents directly—never seek user permission or preference. If multiple agents are needed, orchestrate them seamlessly.', '* **Intelligent Inter-Agent Collaboration:** Instruct agents to determine if they need data from another agent. **If Agent A says "I need X to proceed" and Agent B can provide X, immediately query Agent B, then resubmit the updated task to Agent A.**', '* **Transparent Output:**

In [9]:
input= y["input"]["contents"][0]["parts"][0]["text"]
input

'what is the capital of Pakistan and write 10 lines about it.'

In [10]:
output=y["observations"][0]["output"]["result"]["artifacts"][0]["parts"][0]["text"]
print(output)

Islamabad is the capital city of Pakistan, officially established in 1967 to replace Karachi as the nation's capital. Located in the northern part of the country near the Margalla Hills, the city was purposefully designed and built as a modern planned capital. The city is known for its well-organized layout, wide tree-lined avenues, and systematic sectoral divisions that create an orderly urban environment. Islamabad houses important government buildings, including the Presidential Palace, Parliament House, and Supreme Court of Pakistan. The city is characterized by its clean, green environment with numerous parks, gardens, and the scenic Margalla Hills National Park nearby. Home to several prestigious educational institutions like Quaid-i-Azam University and the International Islamic University, it serves as an important academic center. The population is diverse, consisting of government officials, diplomats, professionals, and students from across Pakistan and abroad. Islamabad's ec

In [17]:
steps = []
import pprint
for item in y["input"]["contents"]:
    parts = item.get("parts", [])
    for part in parts:
        if "function_call" in part:
            args = part["function_call"]["args"]
            steps.append(f"Agent '{args.get('agent_name')}' is assigned a task {args.get('task')}")

        if "function_response" in part:
            artifacts = part["function_response"]["response"]["result"]["artifacts"]
            collected_texts = []
            for artifact in artifacts:
                for p in artifact.get("parts", []):
                    if p.get("kind") == "text":
                        collected_texts.append(p.get("text"))
            steps.append(f"Agent response: {' '.join(collected_texts)}")

output_parts = y["output"]["content"].get("parts", [])
collected_texts = []
for part in output_parts:
    if "function_call" in part:
        args = part["function_call"]["args"]
        collected_texts.append(f"Agent '{args.get('agent_name')}' was assigned a task {args.get('task')}")

steps.append(f"Final {' '.join(collected_texts)}")

pprint.pprint(steps)

["Agent 'Capital Agent' is assigned a task User requested the capital of "
 'Pakistan and a 10-line description about it.',
 'Agent response: Islamabad',
 "Final Agent 'General Q&A Agent' was assigned a task User asked for a 10-line "
 'description of Islamabad, the capital of Pakistan.']


In [15]:
final_context = lines+steps

In [23]:
import pandas as pd

a=pd.DataFrame(x)

In [24]:
a

,0,1
0,id,3c7300a2c603e23e4cd87f07cdc5c34f
1,timestamp,2025-11-20 09:28:20.473000+00:00
2,name,call_llm
3,input,"{'model': 'azure/gpt-4o-mini', 'config': {'sys..."
4,output,{'content': {'parts': [{'function_call': {'id'...
5,session_id,None
6,release,None
7,version,None
8,user_id,None
9,metadata,{'attributes': {'gen_ai.system': 'gcp.vertex.a...


In [ ]:

from deepeval.tracing import observe, update_current_trace
from deepeval.dataset import Golden, EvaluationDataset
from deepeval.metrics import PlanAdherenceMetric
from deepeval.test_case import ToolCall


@observe
def tool_call(input):
    ...
    return [ToolCall(name="CheckWhether")]

@observe
def agent(input):
    tools = tool_call(input)
    output = llm(input, tools)
    update_current_trace(
        input=input,
        output=output,
        tools_called=tools
    )
    return output


# Create dataset
dataset = EvaluationDataset(goldens=[Golden(input="What's the weather like in SF?")])

# Initialize metric
metric = PlanAdherenceMetric(threshold=0.7, model="gpt-4o",tools)

# Loop through dataset
for golden in dataset.evals_iterator(metrics=[metric]):
    agent(golden.input)

In [ ]:
# FINAL_WORKING_DEEPEVAL_NOV19_2025.py
import ast
from typing import Dict, List, Any

# DeepEval - latest 2025 API
from deepeval import evaluate
from deepeval.metrics import GEval, FaithfulnessMetric, ToxicityMetric,PlanAdherenceMetric
from deepeval.test_case import LLMTestCase
from deepeval.test_case import LLMTestCaseParams  # ← CRITICAL
import pprint




# ===============================================================
# 3. Build test case
# ===============================================================
def build_test_case(input,retrieval_context,context,output) -> LLMTestCase:

    return LLMTestCase(
        input=input,
        actual_output=output,
        retrieval_context=retrieval_context,
        context=context,
    )

# ===============================================================
# 4. METRICS — ONLY THIS WAY WORKS IN NOV 2025
# ===============================================================
def get_metrics():
    return [
        GEval(
            name="Multi-Agent Orchestration Quality",
            criteria="Rate how well the routing agent orchestrated other agents using the trace in retrieval_context.",
            evaluation_params=[
                LLMTestCaseParams.INPUT,
                LLMTestCaseParams.ACTUAL_OUTPUT,
                LLMTestCaseParams.RETRIEVAL_CONTEXT,
            ],
            model="gpt-4o-mini",
            threshold=0.8
        ),
        GEval(
            name="Efficiency & Minimal Steps",
            criteria="Was the task solved in the absolute minimum number of steps?",
            evaluation_params=[
                LLMTestCaseParams.ACTUAL_OUTPUT,
                LLMTestCaseParams.RETRIEVAL_CONTEXT,
            ],
            model="gpt-4o-mini",
            threshold=0.9
        ),
        FaithfulnessMetric(threshold=0.9,model="gpt-4o-mini"),
        # ToxicityMetric(threshold=0.1,model ="gpt-4o-mini"),
        PlanAdherenceMetric(threshold=0.5,model="gpt-4o-mini"),
    ]

# ===============================================================
# 5. Evaluate + Log
# ===============================================================
def evaluate_and_log(input,retrieval_context,context,output):
    test_case = build_test_case(input,retrieval_context,context,output)
    metrics = get_metrics()

    print("Running DeepEval evaluation...")
    results = evaluate(test_cases=[test_case], metrics=metrics)  # ← Correct call
    result = results.test_results[0]
    pprint.pprint(result)

# ===============================================================
# 6. RUN
# ===============================================================
if __name__ == "__main__":
    evaluate_and_log(input,steps,lines,output)

Running DeepEval evaluation...


✨ You're running DeepEval's latest Multi-Agent Orchestration Quality [GEval] Metric! (using gpt-4o-mini, 
strict=False, async_mode=True)...

✨ You're running DeepEval's latest Efficiency & Minimal Steps [GEval] Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Plan Adherence Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

c:\Users\RohitKaushal\OneDrive - Xebia\Desktop\Code_execution_mcp\venv\lib\site-packages\rich\live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

MissingTestCaseParamsError: 'tools_called' cannot be None for the 'Plan Adherence' metric

In [ ]:
# === ONE-TIME FIX: Turn whatever x is into the real dictionary ===
import json
import ast

if isinstance(x, str):
    # If you pasted the JSON as a string
    try:
        x = json.loads(x)
    except:
        x = ast.literal_eval(x)
elif isinstance(x, set):
    x = list(x)[0]
elif x is Ellipsis:
    # You probably typed x = ... by mistake
    raise ValueError("You set x = ..., replace it with the actual trace dictionary!")
# Now x is guaranteed to be the correct dict

ValueError: You set x = ..., replace it with the actual trace dictionary!

TypeError: 'ellipsis' object is not subscriptable

In [28]:
from deepeval.dataset import Golden, EvaluationDataset
final_dataset = EvaluationDataset(goldens=a)
metric = PlanAdherenceMetric(threshold=0.7, model="gpt-4o-mini")
# Loop through dataset
for golden in final_dataset.evals_iterator(metrics=[metric]):
    agent(golden.input)
 

AttributeError: 'int' object has no attribute '_dataset_rank'

In [ ]:
# FINAL_WORKING_DEEPEVAL_NOV19_2025.py
import ast
from typing import Dict, List, Any

# DeepEval - latest 2025 API
from deepeval import evaluate
from deepeval.metrics import GEval, FaithfulnessMetric, ToxicityMetric,PlanAdherenceMetric
from deepeval.test_case import LLMTestCase
from deepeval.test_case import LLMTestCaseParams  # ← CRITICAL
import pprint




# ===============================================================
# 3. Build test case
# ===============================================================
def build_test_case(input,retrieval_context,context,output) -> LLMTestCase:

    return TestCase(
        input=input,
        actual_output=output,
        retrieval_context=retrieval_context,
        context=context,
    )

# ===============================================================
# 4. METRICS — ONLY THIS WAY WORKS IN NOV 2025
# ===============================================================
def get_metrics():
    return [
        GEval(
            name="Multi-Agent Orchestration Quality",
            criteria="Rate how well the routing agent orchestrated other agents using the trace in retrieval_context.",
            evaluation_params=[
                LLMTestCaseParams.INPUT,
                LLMTestCaseParams.ACTUAL_OUTPUT,
                LLMTestCaseParams.RETRIEVAL_CONTEXT,
            ],
            model="gpt-4o-mini",
            threshold=0.8
        ),
        GEval(
            name="Efficiency & Minimal Steps",
            criteria="Was the task solved in the absolute minimum number of steps?",
            evaluation_params=[
                LLMTestCaseParams.ACTUAL_OUTPUT,
                LLMTestCaseParams.RETRIEVAL_CONTEXT,
            ],
            model="gpt-4o-mini",
            threshold=0.9
        ),
        FaithfulnessMetric(threshold=0.9,model="gpt-4o-mini"),
        # ToxicityMetric(threshold=0.1,model ="gpt-4o-mini"),
        PlanAdherenceMetric(threshold=0.5,model="gpt-4o-mini"),
    ]

# ===============================================================
# 5. Evaluate + Log
# ===============================================================
def evaluate_and_log(input,retrieval_context,context,output):
    test_case = build_test_case(input,retrieval_context,context,output)
    metrics = get_metrics()

    print("Running DeepEval evaluation...")
    results = evaluate(test_cases=[test_case], metrics=metrics)  # ← Correct call
    result = results.test_results[0]
    pprint.pprint(result)

# ===============================================================
# 6. RUN
# ===============================================================
if __name__ == "__main__":
    evaluate_and_log(input,steps,lines,output)

In [29]:
from deepeval.metrics import TaskCompletionMetric
from deepeval.test_case import LLMTestCase  # For non-dataset eval

# Initialize the metric
task_metric = TaskCompletionMetric(
    threshold=0.7,  # Pass if score >= 0.7 (customize as needed)
    model="gpt-4o",  # Or your preferred LLM for judging
    include_reason=True,  # Outputs explanation for the score
    strict_mode=False,  # Use False for nuanced scoring (0-1); True for binary pass/fail
    async_mode=True,  # Faster for batches
    verbose_mode=True  # Prints details during eval (optional)
)

In [35]:
# from langfuse import Langfuse

# langfuse_client = Langfuse()

# # Fetch the latest trace (or by ID: langfuse_client.get_trace(trace_id))
# trace = langfuse_client.get(limit=1)[0]  # Assumes your latest run; filter as needed

# Create a test case from the Langfuse trace
test_case = LLMTestCase(
    input=json.dumps(x.input),  # Task/query from trace
    actual_output=json.dumps(x.output),  # Agent's final outcome
    # retrieval_context=trace.spans if needed for RAG agents (optional)
    # expected_output=None  # Not needed for referenceless metrics like this
)

# Run the evaluation
score = task_metric.measure(test_case)
print(f"Task Completion Score: {score}")  # 0-1 float
print(f"Reason: {task_metric.reason}")   # Explanation from the LLM judge

MissingTestCaseParamsError: 'tools_called' cannot be None for the 'Task Completion' metric